In [1]:
pip install skmultiflow

ERROR: Could not find a version that satisfies the requirement skmultiflow (from versions: none)
ERROR: No matching distribution found for skmultiflow
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import sys
import time

import numpy as np
import pandas as pd

from datetime import datetime
import json
from statistics import mean

from PCB_iForest_modules.PCB_EIF import *


In [2]:
INF_value = 1519889160.0* 100000

pcb_param_list = {
# PCB-iForest
'window_size' : 200,
'num_trees' : 100,
'extension_level' : 1.0, # in percent of dimension - 1.0 denotes to fully extended version
'anomaly_threshold_eif' : 0.5,
# NDKSWIN-Parameters
'alpha' : 0.01,
'n_dimensions' : 1,
'n_tested_samples' : 0.1,
'stat_size' : 30
}

In [3]:
def load_data(dataFile: str):
    '''
    return data
    @param dataFile: path to the file
    @return: data
    '''
    df = pd.read_csv(dataFile,sep=' ')
    class_labels = df.iloc[:, -1:]
    final_df = df.astype(float)
    data_with_labels = final_df.to_numpy()
    return data_with_labels

In [4]:
def preprocessing_data(data: np.ndarray):
    '''
    handling remaining nan, inf values in data instance; separating the class labels from data instance
    @param data: numpy array
    @return: data, class label
    '''
    data = data.tolist()
    y_label = data[-1]
    if isinstance(y_label, str):
        if y_label !=  "Benign": #anomalous point is represented as 1
            y_label = 1

        else:
            y_label = 0

    x_data = np.asarray(data[:-1])
    x_data = np.nan_to_num(x_data, nan=0.0, posinf= INF_value,
                         neginf=-INF_value)

    return x_data, y_label

In [5]:
def AAR(df: pd.DataFrame):
    df.drop(0,axis=1,inplace=True)
    df.sort_values(by=1,ascending=False,inplace=True)
    df=df.reset_index(drop=True)
    AAR = np.mean(df.index[df[2] == 1].tolist())
    return AAR

In [6]:
dataset_name = "shuttle"
update_type = "U1"   #U1 or U2
D0_path = "./datasets/"+dataset_name+"/"+update_type+"/D0.csv"
deltaD_path = "./datasets/"+dataset_name+"/"+update_type+"/deltaD.csv"

D0 = load_data(D0_path)
deltaD = load_data(deltaD_path)
D0_size = len(D0)
deltaD_size = len(deltaD)

#data = np.concatenate((D0,deltaD))

hyper_parameters = [[128,100]]
params = [128,100]

In [7]:
print("classifier PCBIF processing "+ dataset_name + " with hyper_parameters " + "W" + str(params[0]) + "_T" + str(params[1])+" extension="+str(int(
                                                         pcb_param_list['extension_level'] * D0.shape[1] - 2)))
    

classifier PCBIF processing shuttle with hyper_parameters W128_T100 extension=8


In [8]:
i = 1
random_state = np.random.RandomState(i)
initial_time = time.time()
samples_used = 0
        

In [9]:
ix = np.random.choice(D0_size, size=params[0], replace=False)
D0_sample = D0[ix,:-1]

In [10]:
F = EiForest(D0_sample, ntrees=params[1], sample_size=params[0],
                                         threshold=pcb_param_list['anomaly_threshold_eif'], ExtensionLevel=int(
                                                         pcb_param_list['extension_level'] * D0.shape[1] - 2))

In [11]:
F.ensemble_fit(D0_sample)

In [12]:
ndkswindow = ndk(alpha=pcb_param_list['alpha'], 
                 data=D0_sample, 
                 n_dimensions=pcb_param_list['n_dimensions'], 
                 window_size=params[0], 
                 stat_size=pcb_param_list['stat_size'], 
                 n_tested_samples=pcb_param_list['n_tested_samples'], 
                 fixed_checked_dimension=False, 
                 fixed_checked_sample=False)
        

In [13]:
ix = np.random.choice(deltaD_size, size=params[0], replace=False)
deltaD_sample = deltaD[ix,:-1]

In [14]:
for data_instance in deltaD_sample:
            overall_anomaly_score, individual_anomaly_score = F.compute_paths(data_instance) # self.window[-1]
            F.compare_scores(overall_anomaly_score, individual_anomaly_score)
            #prediction = F.predict_function(overall_anomaly_score)
          

In [15]:
ndkswindow.add_element(deltaD_sample)
if ndkswindow.detected_change():
    update_counter += 1
    F.update_model(deltaD_sample.tolist())
        

In [16]:
D0_ascore = pd.DataFrame(columns=[0,1,2])
for data_instance in D0:
    x_instance, y_instance = preprocessing_data(data_instance)
    overall_anomaly_score, individual_anomaly_score = F.compute_paths(x_instance) # self.window[-1]
    D0_ascore.loc[len(D0_ascore.index)] = [samples_used-1,overall_anomaly_score,y_instance]  
        

In [17]:
deltaD_ascore = pd.DataFrame(columns=[0,1,2])
for data_instance in deltaD:
    x_instance, y_instance = preprocessing_data(data_instance)
    overall_anomaly_score, individual_anomaly_score = F.compute_paths(x_instance) # self.window[-1]
    deltaD_ascore.loc[len(deltaD_ascore.index)] = [samples_used-1,overall_anomaly_score,y_instance]
        

In [18]:
print(AAR(D0_ascore),AAR(deltaD_ascore))

5648.573033707865 33.541666666666664
